In [2]:
import os
from tqdm import tqdm_notebook as tqdm
import numpy as np

root = 'data'
features = []
name = []
for subdir_entry in os.scandir(root):
    if not subdir_entry.is_dir():
        continue
    subdir_path = os.path.join(subdir_entry.path, 'KEYPOINT_DIR')
    if not os.path.exists(subdir_path):
        continue

    try:
        for f_entry in os.scandir(subdir_path):
            if not f_entry.is_dir():
                continue

            feature = []
            npy_files = sorted([
                entry.path for entry in os.scandir(f_entry.path)
                if entry.is_file() and entry.name.endswith('.npy')
            ])
            for npy_path in tqdm(npy_files):
                feature.append(np.load(npy_path, allow_pickle=False))  # safer
            name.append(f_entry.path)

            if feature:
                features.append(np.array(feature))
    except Exception as e:
        print(f"Skipping {subdir_entry.name} due to error: {e}")


/tmp/ipykernel_1747/2553932779.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for npy_path in tqdm(npy_files):


  0%|          | 0/153 [00:00<?, ?it/s]

  0%|          | 0/151 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/69 [00:00<?, ?it/s]

...

  0%|          | 0/82 [00:00<?, ?it/s]

  0%|          | 0/105 [00:00<?, ?it/s]

  0%|          | 0/103 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [3]:
len(features),len(name)

(16, 16)

In [4]:
for i,j in zip(features,name):
    print(i.shape,j)

(153, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0001
(151, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0002
(10, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0003
(69, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0004
(238, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0005
(98, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0006
(32, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0007
(138, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0008
(49, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0009
(89, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0010
(120, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0011
(82, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0012
(105, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0013
(103, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0014
(5, 17, 3) data/Gerrit_Cole_CH_videos_4S/KEYPOINT_DIR/pitch_0015
(11,

In [5]:
import os
import pandas as pd

def extract_label_from_path(j):
    """
    從 keypoint 資料夾路徑中取出球員名稱、影片名稱、對應 CSV、取得描述
    """
    # 解析球員與影片路徑
    parts = j.split(os.sep)  # 例如 ['data', 'Gerrit_Cole_CH_videos_4S', 'KEYPOINT_DIR', 'pitch_0001']
    player_folder = parts[1]  # 'Gerrit_Cole_CH_videos_4S'
    video_name = parts[-1] + '.mp4'  # 'pitch_0001.mp4'

    # 取得球員名：去掉尾巴的 "_videos_4S"
    player_name = player_folder.replace("_videos_4S", "")
    csv_path = os.path.join('data', player_folder, f"{player_name}.csv")

    # 讀取 CSV 並找對應 label
    if not os.path.exists(csv_path):
        print(f"❌ CSV 檔案不存在: {csv_path}")
        return None

    df = pd.read_csv(csv_path)

    # 忽略大小寫來比對
    matched_row = df[df['Filename'].str.lower() == video_name.lower()]
    if matched_row.empty:
        print(f"⚠️ 找不到對應影片: {video_name} in {csv_path}")
        return None

    return matched_row.iloc[0]['description']


In [6]:
y = []

for j in name:
    label = extract_label_from_path(j)
    if label is not None:
        y.append(label)
    else:
        y.append('Unknown')  # 或是跳過這筆


In [7]:
for i,j in zip(features,y):
    print(i.shape,j)

(153, 17, 3) ball
(151, 17, 3) foul
(10, 17, 3) hit_into_play
(69, 17, 3) hit_into_play
(238, 17, 3) ball
(98, 17, 3) foul
(32, 17, 3) foul
(138, 17, 3) called_strike
(49, 17, 3) ball
(89, 17, 3) ball
(120, 17, 3) called_strike
(82, 17, 3) ball
(105, 17, 3) ball
(103, 17, 3) foul
(5, 17, 3) ball
(11, 17, 3) ball
